# [STARTER] Udaplay Project

## Part 01 - Offline RAG

In this part of the project, you'll build your VectorDB using Chroma.

The data is inside folder `project/starter/games`. Each file will become a document in the collection you'll create.
Example.:
```json
{
  "Name": "Gran Turismo",
  "Platform": "PlayStation 1",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment",
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "YearOfRelease": 1997
}
```


### Setup

In [ ]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [1]:
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv

In [ ]:
# TODO: Create a .env file with the following variables
# OPENAI_API_KEY="YOUR_KEY"
# CHROMA_OPENAI_API_KEY="YOUR_KEY"
# TAVILY_API_KEY="YOUR_KEY"

In [6]:
# TODO: Load environment variables
load_dotenv()

True

### VectorDB Instance

In [11]:
# TODO: Instantiate your ChromaDB Client
# Choose any path you want
chroma_client = chromadb.PersistentClient(path="chromadb")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### Collection

In [12]:
# TODO: Pick one embedding function
# If picking something different than openai, 
# make sure you use the same when loading it
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(OPENAI_API_KEY)

In [14]:
# TODO: Create a collection
# Choose any name you want
collection = chroma_client.create_collection(
    name="udaplay",
    embedding_function=embedding_fn
)

### Add documents

In [21]:
# Make sure you have a directory "project/starter/games"
data_dir = "games"
documents_added=0

for file_name in sorted(os.listdir(data_dir)):
    if not file_name.endswith(".json"):
        continue

    file_path = os.path.join(data_dir, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        game = json.load(f)

    # You can change what text you want to index
    content = f"[{game['Platform']}] {game['Name']} ({game['YearOfRelease']}) - {game['Description']}"

    # Use file name (like 001) as ID
    doc_id = os.path.splitext(file_name)[0]

    collection.add(
        ids=[doc_id],
        documents=[content],
        metadatas=[game]
    )
    
    documents_added += 1
    print(f"Added document {doc_id}: {game['Name']}")

print(f"\nTotal documents added: {documents_added}")
print(f"Collection count: {collection.count()}")

# Test a simple query to verify the collection works
test_results = collection.query(
    query_texts=["action game"],
    n_results=3
)
print(f"\nTest query results: Found {len(test_results['ids'][0])} games matching 'action game'")
for i, doc_id in enumerate(test_results['ids'][0]):
    print(f"  - {doc_id}: {test_results['metadatas'][0][i]['Name']}")

Added document 001: Gran Turismo
Added document 002: Grand Theft Auto: San Andreas
Added document 003: Gran Turismo 5
Added document 004: Marvel's Spider-Man
Added document 005: Marvel's Spider-Man 2
Added document 006: Pokémon Gold and Silver
Added document 007: Pokémon Ruby and Sapphire
Added document 008: Super Mario World
Added document 009: Super Mario 64
Added document 010: Super Smash Bros. Melee
Added document 011: Wii Sports
Added document 012: Mario Kart 8 Deluxe
Added document 013: Kinect Adventures!
Added document 014: Minecraft
Added document 015: Halo Infinite

Total documents added: 15
Collection count: 15

Test query results: Found 3 games matching 'action game'
  - 004: Marvel's Spider-Man
  - 002: Grand Theft Auto: San Andreas
  - 013: Kinect Adventures!
